In [ ]:
import functions_framework
import pandas as pd
from datetime import datetime, timedelta
import requests
from pytz import timezone 
from datetime import datetime
from keys import gcp
from keys import AeroDatabox
import sqlalchemy



 #USE YOUR OWN OPEN WHEATHER KEY HERE

def get_flight_data(request):

    
    schema = "gans_global" # The name of your database
    host = "35.187.50.113"
    user = "root"
    password = gcp # Your gcp password
    port = 3306

    connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

   


    # As we will be working in the cloud, computers can be located anywhere
    # Let's adjust the computer timezone to our local timezone

    
    api_key = AeroDatabox


    berlin_timezone = timezone('Europe/Berlin')
    today = datetime.now(berlin_timezone).date()
    tomorrow = (today + timedelta(days=1))

    flight_items = []
    icao_list = ['EDDH']
    for icao in icao_list:
        # the api can only make 12 hour calls, therefore, 2 12 hour calls make a full day
        # using the nested lists below we can make a morning call and extract the data
        # then make an afternoon call and extract the data
        times = [["00:00","11:59"],
                 ["12:00","23:59"]]

        for time in times:

            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"

            querystring = {"withLeg":"true",
                        "direction":"Arrival",
                        "withCancelled":"false",
                        "withCodeshared":"true",
                        "withCargo":"false",
                        "withPrivate":"false"}

            headers = {
              'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
              'x-rapidapi-key': api_key
              }

            response = requests.get(url, headers=headers, params=querystring)

            flights_json = response.json()

            retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")

            for item in flights_json["arrivals"]:
                flight_item = {
                    "arrival_airport_icao": icao,
                    "departure_airport_icao": item["departure"]["airport"].get("icao", None),
                    "departure_airport_name": item["departure"]["airport"].get("name", None),
                    "scheduled_arrival_time": item["arrival"]["scheduledTime"].get("local", None),
                    "flight_number": item.get("number", None),
                    "data_retrieved_at": retrieval_time
                }

                flight_items.append(flight_item)

    flights= pd.DataFrame(flight_items)
    flights["scheduled_arrival_time"] = flights["scheduled_arrival_time"].str[:-6]
    flights["scheduled_arrival_time"] = pd.to_datetime(flights["scheduled_arrival_time"])
    flights["data_retrieved_at"] = pd.to_datetime(flights["data_retrieved_at"])

    flights.to_sql('flights',
                    if_exists='append',
                    con=connection_string,
                    index=False)
                    
    return 'Data has been updated'

    